## Data processing

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import sys
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

### Train data

In [3]:
index_columns = ["Unit", "Cycle"]
settings_columns = ["setting" + str(i + 1) for i in range(3)]
sensor_columns = ["s" + str(i + 1) for i in range(21)]
final_columns = index_columns + settings_columns + sensor_columns
engine_data = pd.read_csv("../input/cmapass/train_FD001.txt", header=None, sep=" ")
engine_data = engine_data[engine_data.columns[0:26]]
engine_data.columns = final_columns

engine_data

In [4]:


engine_data.copy(deep=True).drop(['Unit'], axis=1).describe()


In [5]:


engine_data.copy(deep=True).drop(['Unit'], axis=1).hist()

### Test data

In [6]:
engine_data_test = pd.read_csv("../input/cmapass/test_FD001.txt", header=None, sep=" ")
engine_data_test = engine_data_test[engine_data_test.columns[0:26]]
engine_data_test.columns = final_columns

engine_data_test

In [7]:
engine_data[engine_data['Unit'] == 1].head()

In [8]:
engine_data[engine_data['Unit'] == 1].tail()

In [9]:


def get_RUL(engine_data):
    
    last_cycle = engine_data.groupby(['Unit'])["Cycle"].max().reset_index()
    
   
    final = pd.merge(engine_data, last_cycle, how="inner", on=["Unit"])
    
   
    engine_data["RUL"] = final["Cycle_y"] - final["Cycle_x"]
    
   
    engine_data["binary_class"] = engine_data["RUL"].map(lambda x: 1 if x <= 30 else 0)
    
    
    engine_data["multi_class"] = engine_data["RUL"].map(lambda x: 2 if x <= 15 else 1 if x<= 30 else 0)
    return engine_data

train_labels = get_RUL(engine_data.copy(deep=True))
RUL_train = train_labels["RUL"]
binary_class_train = train_labels["binary_class"]
multi_class_train = train_labels["multi_class"]

train_labels

In [10]:

remaining_RUL_test = pd.read_csv("../input/cmapass/RUL_FD001.txt", header=None)
remaining_RUL_test["Unit"] = remaining_RUL_test.index + 1


def get_RUL_test(engine_data_test, remaining_RUL_test):
    
    last_cycle = engine_data_test.copy(deep=True).groupby(['Unit'])["Cycle"].max().reset_index()
    
  
    final = pd.merge(last_cycle, engine_data_test.copy(deep=True), how="inner", on=["Unit", "Cycle"])
    final = pd.merge(final, remaining_RUL_test, how="inner", on=["Unit"])
    final["RUL"] = final[0]
  
    final["binary_class"] = final["RUL"].map(lambda x: 1 if x <= 30 else 0)
   
    final["multi_class"] = final["RUL"].map(lambda x: 2 if x <= 15 else 1 if x<= 30 else 0)
    return final

test_labels = get_RUL_test(engine_data_test.copy(deep=True), remaining_RUL_test)
RUL_test = test_labels["RUL"]
binary_class_test = test_labels["binary_class"]
multi_class_test = test_labels["multi_class"]

test_labels

## Exploratory Visualization

In [11]:

target_value = ['RUL']
plot_data = engine_data.copy(deep=True)
plot_data["RUL"] = RUL_train
for i in range(24):
    graph = sns.pairplot(plot_data[plot_data["Unit"] <= 15], x_vars=target_value, 
                         y_vars=(sensor_columns + settings_columns)[i], hue="Unit", size=3, aspect=2.5)
    graph = graph.map(plt.scatter)
    graph = graph.set(xlim=(300,0))
    graph = graph.add_legend()

# Feature Transformation

In [12]:


from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA


features_dropped = ['Unit', 'setting1', 'setting2', 'setting3', 's1', 's5', 's10', 's16', 's18', 's19']

normalized_data = engine_data.copy(deep=True).drop(features_dropped, axis=1)

normalized_data = np.log(normalized_data)

n_components = 13


pca = PCA(n_components=n_components)
reduced_data = pca.fit_transform(normalized_data)
reduced_data = pd.DataFrame(reduced_data, columns=['Dimension ' + str(i+1) for i in range(n_components)])

variance = pd.DataFrame(pca.explained_variance_ratio_)
print(np.cumsum(pca.explained_variance_ratio_))


last_cycle = engine_data_test.copy(deep=True).groupby(['Unit'])["Cycle"].max().reset_index()
normalized_data_test = engine_data_test.copy(deep=True)
normalized_data_test = pd.merge(last_cycle, normalized_data_test, how="inner", on=["Unit", "Cycle"])
normalized_data_test = normalized_data_test.drop(features_dropped, axis=1)

normalized_data_test = np.log(normalized_data_test)


reduced_data_test = pca.transform(normalized_data_test)
reduced_data_test = pd.DataFrame(reduced_data_test, columns=['Dimension ' + str(i+1) for i in range(n_components)])
print(reduced_data)
print(reduced_data_test)

## Train and evaluate the model

### Predict RUL using Regression

In [13]:
# RUL Model selection (Regression)

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge, HuberRegressor, Lasso
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR  
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor



def analyse_regression_models(train, labels):
    results = {}
    
    def test_model(clf):
        cv = KFold(n_splits=2, shuffle=True, random_state=20)
        mse = make_scorer(mean_squared_error)
        mse_val_score = cross_val_score(clf, train, labels, cv=cv, scoring=mse)
        scores = [mse_val_score.mean()]
        return scores
    
    clf = LinearRegression()
    results["Linear"] = test_model(clf)
    
    clf = Ridge()
    results["Ridge"] = test_model(clf)
    
    clf = BayesianRidge()
    results["BayesianRidge"] = test_model(clf)
    
    clf = HuberRegressor()
    results["Huber"] = test_model(clf)
    
    clf = Lasso()
    results["Lasso"] = test_model(clf)
    
    clf = MLPRegressor()
    results["Neural Network"] = test_model(clf)
    
    clf = BaggingRegressor()
    results["Bagging"] = test_model(clf)
    
    clf = RandomForestRegressor()
    results["RandomForest"] = test_model(clf)
    
    clf = AdaBoostRegressor()
    results["AdaBoost"] = test_model(clf)
    
    clf = SVR()
    results["SVM"] = test_model(clf)
    
    results = pd.DataFrame.from_dict(results, orient='index')
    results.columns = ["Mean Squared Error"] 
    results = results.sort_values(by=["Mean Squared Error"], ascending=True)
    results.plot(kind="bar", title="Model Scores")
    axes = plt.gca()
    axes.set_ylim([800, 2500])
    return results

analyse_regression_models(reduced_data, RUL_train)

In [14]:


from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.neural_network import MLPRegressor

cv = KFold(n_splits=2, shuffle=True, random_state=20)


parameters = {
    'hidden_layer_sizes': [105],
    'activation': ['logistic', 'relu'],
    'solver': ['sgd'],
    'learning_rate': ['constant', 'adaptive'],
    'learning_rate_init': [0.001],
    'momentum': [0.95],
    'batch_size': [100]
}


clf = MLPRegressor(random_state=20)
mse = make_scorer(mean_squared_error)


grid_obj = GridSearchCV(clf, parameters, cv=cv, scoring=mse)


grid_fit = grid_obj.fit(reduced_data, RUL_train)


best_clf = grid_fit.best_estimator_ 
print(best_clf)

best_clf.fit(reduced_data, RUL_train)

RUL_predictions = best_clf.predict(reduced_data_test)

print("Mean Absolute error: " + str(mean_absolute_error(RUL_predictions, RUL_test)))
print("Mean Squared error: " + str(mean_squared_error(RUL_predictions, RUL_test)))


plt.scatter(RUL_test, RUL_predictions)

straight_line = np.arange(0, 400)
plt.plot(straight_line, straight_line)
plt.title("Fitted Values")
print(RUL_predictions)

### Predict if an engine will fail within x1 cycles using Binary Classification

Value of x1 is 30 in this case

In [15]:


from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis



def analyse_binary_classification_models(train, labels):
    results = {}
    
    def test_model(clf):
        cv = KFold(n_splits=2, shuffle=True, random_state=20)
        accuracy = make_scorer(accuracy_score)
        accuracy_val_score = cross_val_score(clf, train, labels, cv=cv, scoring=accuracy)
        scores = [accuracy_val_score.mean()]
        return scores
    
    clf = KNeighborsClassifier(3)
    print("KNeighbors")
    results["KNeighbors"] = test_model(clf)
    
    clf = SVC()
    print("SVC")
    results["SVM"] = test_model(clf)
    
    clf = GaussianProcessClassifier()
    print("GaussianProcess")
    results["GaussianProcess"] = test_model(clf)
    
    clf = GaussianNB()
    print("GaussianNB")
    results["GaussianNB"] = test_model(clf)
    
    clf = QuadraticDiscriminantAnalysis()
    print("QuadraticDiscriminantAnalysis")
    results["QuadraticDiscriminantAnalysis"] = test_model(clf)
    
    clf = MLPClassifier(alpha=1)
    print("Neural Network")
    results["Neural Network"] = test_model(clf)
    
    clf = AdaBoostClassifier()
    print("AdaBoost")
    results["AdaBoost"] = test_model(clf)
    
    clf = DecisionTreeClassifier(max_depth=5)
    print("Decision Tree")
    results["Decision Tree"] = test_model(clf)
    
    clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
    print("RandomForest")
    results["RandomForest"] = test_model(clf)
    
    results = pd.DataFrame.from_dict(results, orient='index')
    results.columns = ["Accuracy"] 
    results = results.sort_values(by=["Accuracy"], ascending=False)
    results.plot(kind="bar", title="Model Scores")
    axes = plt.gca()
    axes.set_ylim([0, 1])
    return results

analyse_binary_classification_models(reduced_data, binary_class_train)

In [16]:


from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier


cv = KFold(n_splits=2, shuffle=True, random_state=20)


parameters = {
    'learning_rate': [0.7, 1],
    'algorithm': ['SAMME', 'SAMME.R'],
    'n_estimators': [12, 25, 50, 100]
}


clf = AdaBoostClassifier(random_state=20)

accuracy = make_scorer(accuracy_score)


grid_obj = GridSearchCV(clf, parameters, cv=cv, scoring=accuracy)


grid_fit = grid_obj.fit(reduced_data, binary_class_train)


best_clf = grid_fit.best_estimator_ 
print(best_clf)

best_clf.fit(reduced_data, binary_class_train)

binary_class_predictions = best_clf.predict(reduced_data_test)

print("Accuracy: " + str(accuracy_score(binary_class_predictions, binary_class_test)))
print(binary_class_predictions)


### Predict if an engine will fail in different time periods using Multi class classification 
In this case, the 3 classes are RUL <=15, 15 < RUL <= 30 and RUL > 30

In [17]:


from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multiclass import OneVsRestClassifier



def analyse_multiclass_classification_models(train, labels):
    results = {}
    
    def test_model(clf):
        clf = OneVsRestClassifier(clf)
        cv = KFold(n_splits=2, shuffle=True, random_state=20)
        accuracy = make_scorer(accuracy_score)
        accuracy_val_score = cross_val_score(clf, train, labels, cv=cv, scoring=accuracy)
        scores = [accuracy_val_score.mean()]
        return scores
    
    clf = KNeighborsClassifier(3)
    print("KNeighbors")
    results["KNeighbors"] = test_model(clf)
    
    clf = SVC()
    print("SVC")
    results["SVM"] = test_model(clf)
    
    clf = GaussianProcessClassifier()
    print("GaussianProcess")
    results["GaussianProcess"] = test_model(clf)
    
    clf = GaussianNB()
    print("GaussianNB")
    results["GaussianNB"] = test_model(clf)
    
    clf = QuadraticDiscriminantAnalysis()
    print("QuadraticDiscriminantAnalysis")
    results["QuadraticDiscriminantAnalysis"] = test_model(clf)
    
    clf = MLPClassifier(alpha=1)
    print("Neural Network")
    results["Neural Network"] = test_model(clf)
    
    clf = AdaBoostClassifier()
    print("AdaBoost")
    results["AdaBoost"] = test_model(clf)
    
    clf = DecisionTreeClassifier(max_depth=5)
    print("Decision Tree")
    results["Decision Tree"] = test_model(clf)
    
    clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
    print("RandomForest")
    results["RandomForest"] = test_model(clf)
    
    results = pd.DataFrame.from_dict(results, orient='index')
    results.columns = ["Accuracy"] 
    results = results.sort_values(by=["Accuracy"], ascending=False)
    results.plot(kind="bar", title="Model Scores")
    axes = plt.gca()
    axes.set_ylim([0, 1])
    return results

analyse_multiclass_classification_models(reduced_data, multi_class_train)

In [18]:


from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier


cv = KFold(n_splits=2, shuffle=True, random_state=20)

parameters = {
    'learning_rate': [0.7, 1],
    'algorithm': ['SAMME', 'SAMME.R'],
    'n_estimators': [200, 50]
}


clf = AdaBoostClassifier(random_state=20)

accuracy = make_scorer(accuracy_score)


grid_obj = GridSearchCV(clf, parameters, cv=cv, scoring=accuracy)


grid_fit = grid_obj.fit(reduced_data, multi_class_train)


best_clf = grid_fit.best_estimator_ 
best_clf = OneVsRestClassifier(best_clf)
print(best_clf)

best_clf.fit(reduced_data, multi_class_train)

multi_class_predictions = best_clf.predict(reduced_data_test)

print("Accuracy: " + str(accuracy_score(multi_class_predictions, multi_class_test)))
print(multi_class_predictions)